In [242]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

In [243]:
#load pcd file, filter, downsample
#pcdn = o3d.io.read_point_cloud("loop.pcd")
voxel_size = 0.4
pcd = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
pcd.estimate_normals()
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcd.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd])

In [244]:
def filter_by_normal(pcd, hight_variance = 0.01):
    ind = []
    for i in range(len(pcd.points)):
        if ((np.abs(pcd.normals[i][2]) < 0.3) and ((np.abs(pcd.normals[i][0]) > 0.8) or (np.abs(pcd.normals[i][1]) > 0.8))):
            pcd.points[i][2] = np.random.rand()*hight_variance
        else:
            ind.append(i)
    result = pcd.select_by_index(ind, invert=True)
    return result

In [245]:
def are_vectors_perpendicular(v1, v2, threshold):
    
    # Normalize the vectors to unit length
    v1_u = v1 / np.linalg.norm(v1)
    v2_u = v2 / np.linalg.norm(v2)
    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    angle = np.degrees(angle)
    
    if (180 <= angle < 360):
        angle -= 180
        
    if ((90-threshold) <= angle <= (90+threshold)):
        return True
    else:
        return False


In [246]:
pcd_flat = filter_by_normal(pcd)

#o3d.visualization.draw_geometries([pcd_flat])

In [247]:
oboxes = pcd_flat.detect_planar_patches(
normal_variance_threshold_deg=50,
coplanarity_deg=85,
outlier_ratio=0.75,
min_plane_edge_length=2,
min_num_points=10,
search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

print("Detected {} patches".format(len(oboxes)))

geometries = []
meshes = []
for obox in oboxes:

    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox, scale=[1, 1, 0.0001])
    mesh.paint_uniform_color(obox.color)
    mesh.compute_triangle_normals()
    mesh.compute_vertex_normals()
    normals = np.asarray(mesh.vertex_normals)

    
    meshes.append(mesh)
    geometries.append(obox)



Detected 24 patches


In [248]:
def devide_meshes_hor_ver(meshes):
    
    first = meshes[0]
    tmp = False
    ver_patches = []
    hor_patches = []
    if (are_vectors_perpendicular(first.vertex_normals[0], np.asarray([1,0,0]), 15)):
        ver_patches.append(first)
    else:
        hor_patches.append(first)
        tmp = True
    

    for i in range(1,len(meshes)):
        patch_normal = meshes[i].vertex_normals[0]

        if (are_vectors_perpendicular(first.vertex_normals[0], patch_normal, 10)):
            if (tmp):
                
                ver_patches.append(meshes[i])
            else:
                hor_patches.append(meshes[i])
        else:
            if (tmp):
                hor_patches.append(meshes[i])
            else:
                ver_patches.append(meshes[i])



    return hor_patches, ver_patches


In [249]:
hor_patches, ver_patches = devide_meshes_hor_ver(meshes)
o3d.visualization.draw_geometries(ver_patches)
o3d.visualization.draw_geometries(hor_patches)
print(len(hor_patches), len(ver_patches))

6 18


In [250]:
def get_mesh_distance(mesh1, mesh2, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    bb1 = mesh1.get_oriented_bounding_box()
    bb1_center = bb1.get_center()[1-o[orientation]]
    bb2 = mesh2.get_oriented_bounding_box()
    bb2_center = bb2.get_center()[1-o[orientation]]
    
    
    if (bb1_center < 0 < bb2_center):
        dist = -bb1_center + bb2_center
    elif (bb2_center < 0 < bb1_center):
        dist = bb1_center - bb2_center
    else:
        dist = np.abs(bb1_center-bb2_center)
        
    return dist

In [251]:
def mesh_correspondance(mesh1, mesh2, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    
    bb1 = mesh1.get_oriented_bounding_box()
    bb2 = mesh2.get_oriented_bounding_box()
    bb1_min = bb1.get_min_bound()
    bb1_max = bb1.get_max_bound()
    bb2_min = bb2.get_min_bound()
    bb2_max = bb2.get_max_bound()
    bb1_center = bb1.get_center()[o[orientation]]
    bb2_center = bb2.get_center()[o[orientation]]
    
    bb1_center2 = bb1.get_center()[1-o[orientation]]
    bb2_center2 = bb2.get_center()[1-o[orientation]]
    
    if (bb1_center2 < 0 < bb2_center2):
        dist = -bb1_center2 + bb2_center2
    elif (bb2_center2 < 0 < bb1_center2):
        dist = bb1_center2 - bb2_center2
    else:
        dist = np.abs(bb1_center2-bb2_center2)
    
    if (dist < 1):
        return False
    
    if (bb1_min[o[orientation]] < bb2_center < bb1_max[o[orientation]]):
        return True
    if (bb2_min[o[orientation]] < bb1_center < bb2_max[o[orientation]]):

        return True
    
    
    return False
    

In [252]:
def find_nearest_mesh(mesh1, meshes, orientation):
    o = {
  "vertical": 0,
  "horizontal": 1}
    dist = np.Inf
    index = 0
    
    bb1 = mesh1.get_oriented_bounding_box()
    bb1_center = bb1.get_center()[1-o[orientation]]
    
    for mesh2, i in zip(meshes, range(len(meshes))):
        
        bb2 = mesh2.get_oriented_bounding_box()
        bb2_center = bb2.get_center()[1-o[orientation]]
        
        if not (mesh_correspondance(mesh1, mesh2, orientation)):
            continue
        
        
        if (bb1_center < 0 < bb2_center):
            dist_tmp = -bb1_center + bb2_center
        elif (bb2_center < 0 < bb1_center):
            dist_tmp = bb1_center - bb2_center
        else:
            dist_tmp = np.abs(bb1_center-bb2_center)

        if dist_tmp == 0:
            continue
        if dist_tmp < dist:
            dist = dist_tmp
            index = i
    
    return dist, index
    
    

In [254]:
def groud_corresponding_meshes(corr, corr_tuples, patches,  orientation):
    
    
    for i in range(len(patches)):
        mesh1 = patches[i]
        for j in range(i+1, len(patches)):
            mesh2 = patches[j]
            nearest_dist, _  = find_nearest_mesh(mesh1, patches, orientation)
        
      
            if (nearest_dist < get_mesh_distance(mesh1, mesh2, orientation)):
                continue
        
            if (mesh_correspondance(mesh1, mesh2, orientation)):
                color = np.random.rand(3)
                mesh1.paint_uniform_color(color)
                mesh2.paint_uniform_color(color)
            
                corr.append(mesh1)
                corr.append(mesh2)
                corr_tuples.append((mesh1, mesh2))
    return corr, corr_tuples


In [255]:


corr, corr_tuples = groud_corresponding_meshes([],[],hor_patches, "horizontal")

corr, corr_tuples = groud_corresponding_meshes(corr, corr_tuples, ver_patches, "vertical")


In [256]:
def create_box_at_point(point):
    
    box = o3d.geometry.TriangleMesh.create_box(0.1,0.1,0.1)
    box.paint_uniform_color([0,1,0])
    
    box.translate(point, False)
    
    
    return box

In [257]:
def find_midpoint_between_planes(plane1, plane2):
    center1 = plane1.get_center()
    center2 = plane2.get_center()

    vec_1_2 = center2 - center1

    midpoint = center1 + vec_1_2 / 2
    return midpoint

In [258]:
#midpoints = []
#marker_meshes = []
#for tup in corr_tuples:
#    midpoint = find_midpoint_between_planes(tup[0], tup[1])
#    
#    dist = np.inf
#    for point in midpoints:
#        tmp = np.linalg.norm(midpoint-point)
#        if tmp < dist:
#            dist = tmp 
#    print(dist)
#    if dist > 1.5:
#        midpoints.append(midpoint)
#        marker_meshes.append(create_box_at_point(midpoint))
#
#print(midpoints)
#o3d.visualization.draw_geometries(corr + marker_meshes)

In [271]:
midpoints = []
marker_meshes = []

def find_midpoints(midpoints, marker_meshes):
    
    for tup in corr_tuples:
        midpoint = find_midpoint_between_planes(tup[0], tup[1])
    
    
        midpoints.append(midpoint)
        marker_meshes.append(create_box_at_point(midpoint))
    return midpoints, marker_meshes

find_midpoints(midpoints, marker_meshes)

print(midpoints)
o3d.visualization.draw_geometries(corr + marker_meshes)

[array([-7.07499635,  5.39886525,  0.00730267]), array([-7.43332267e+00,  4.50392608e+00,  6.71971015e-03]), array([-3.93398394e+01,  3.83754027e-01,  4.80614041e-03]), array([-4.26263079e+01,  6.91182629e-01,  5.51489984e-03]), array([-2.29134496e+01,  2.66680455e-01,  4.99871347e-03]), array([-1.47350283e+01,  8.21848818e-02,  5.26905889e-03]), array([-1.84104219, -0.18987515,  0.00511314]), array([-1.09687860e+01, -2.96888832e-03,  5.35368714e-03]), array([-3.39391230e+01, -2.80626851e+00,  1.59059086e-03]), array([-3.50356682e+01, -3.07202217e+00,  1.95928900e-03])]


In [260]:
def create_uniform_pc_from_bb(bb_axis, voxel_size, color):
    pc = o3d.geometry.PointCloud()
    bb = o3d.geometry.OrientedBoundingBox.create_from_axis_aligned_bounding_box(bb_axis)
    mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(bb)
    mesh = mesh.paint_uniform_color([0,1,0])
    vg = o3d.geometry.VoxelGrid.create_from_triangle_mesh(mesh, voxel_size)
    voxels = vg.get_voxels()
    grid_indexes = [x.grid_index for x in voxels]
    
    voxel_centers = [vg.get_voxel_center_coordinate(index) for index in grid_indexes]
    
    pc.points.extend(o3d.utility.Vector3dVector(voxel_centers))  
    pc.paint_uniform_color(color)
    return pc



In [261]:
def hull_to_uniform_pc(hull, voxel_size, color):
    pc = o3d.geometry.PointCloud()
    
    vg = o3d.geometry.VoxelGrid.create_from_triangle_mesh(hull, voxel_size)
    voxels = vg.get_voxels()
    grid_indexes = [x.grid_index for x in voxels]
    
    voxel_centers = [vg.get_voxel_center_coordinate(index) for index in grid_indexes]
    
    pc.points.extend(o3d.utility.Vector3dVector(voxel_centers))  
    pc.paint_uniform_color(color)
    return pc


In [262]:
def create_uniform_pc(pcd, voxel_size, color):
    pc = o3d.geometry.PointCloud()
    
    vg = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size)
    voxels = vg.get_voxels()
    grid_indexes = [x.grid_index for x in voxels]
    
    voxel_centers = [vg.get_voxel_center_coordinate(index) for index in grid_indexes]
    
    pc.points.extend(o3d.utility.Vector3dVector(voxel_centers))  
    pc.paint_uniform_color(color)
    return pc


In [263]:
#bb = pcd_flat.get_axis_aligned_bounding_box()
#bb.color = np.asarray([0,0,1])
#uniform_pc = create_uniform_pc_from_bb(bb, 0.2, [1,0,0])
hull, _ = pcd_flat.compute_convex_hull()

uniform_pc = hull_to_uniform_pc(hull, 0.2, [1,0,0])

o3d.visualization.draw_geometries([pcd_flat, uniform_pc, bb])

pcd_flat = create_uniform_pc(pcd_flat, 0.1, [0,0,0])
o3d.visualization.draw_geometries([pcd_flat])

In [264]:
def find_closest_vector_index(input_vector, pc):
    # Convert the input vector and vector list to numpy arrays for easier calculations
    input_vector = np.array(input_vector)
    vector_list = np.asarray(pc.points)

    # Calculate the Euclidean distances between the input vector and all vectors in the list
    distances = np.linalg.norm(vector_list - input_vector, axis=1)

    # Find the index of the vector with the minimum distance
    closest_index = np.argmin(distances)

    return closest_index

In [265]:
def knn_search_pointclouds(tree_grid, tree_target, center, radius):
    

    [ka, idxa, _] = tree_grid.search_radius_vector_3d(center, radius)
    
    [kb, idxb, _] = tree_target.search_radius_vector_3d(center, radius)
    
    return ka, kb, idxa, idxb
    

In [266]:
def get_ids_in_direction(idx_grid, idx_target, coords_grid, coords_target, center, direction):
    
   

    d = {
    "up": (1,False),
    "down": (1,True),
    "left": (0,True),
    "right": (0,False)}
    
    xory = d[direction][0]
    smaller = d[direction][1]
    
    idx_dir_grid = []
    idx_dir_target = []
    for i in range(len(idx_grid)):
        if smaller:
            if coords_grid[i][xory] < center[xory]:
                idx_dir_grid.append(idx_grid[i])
        else:
            if coords_grid[i][xory] > center[xory]:
                idx_dir_grid.append(idx_grid[i])
    
    for i in range(len(idx_target)):
        if smaller:
            if coords_target[i][xory] < center[xory]:
                idx_dir_target.append(idx_target[i])
        else:
            if coords_target[i][xory] > center[xory]:
                idx_dir_target.append(idx_target[i])
            
            
    return (idx_dir_grid, idx_dir_target)
        

In [297]:
# search multiple iterations until boundary found
search_radius = np.ceil(np.sqrt((voxel_size*2)**2+(voxel_size*2)**2)*10000)/10000

uniform_pc.paint_uniform_color([1,0,0])
pcd_flat.paint_uniform_color([0,0,0])

PointCloud with 4068 points.

In [298]:
def grow_void(pcd_grid, pcd_target, initial_seed, initial_set=set(), search_radius=0.2, stop_threshold=2, color=[0,0,1]):


    tree_grid = o3d.geometry.KDTreeFlann(pcd_grid)
    tree_target = o3d.geometry.KDTreeFlann(pcd_target)


    #initial search
    index = find_closest_vector_index(initial_seed, pcd_grid)
    center = pcd_grid.points[index]
    k_grid, k_target, idxu, idxf = knn_search_pointclouds(tree_grid, tree_target, center, search_radius)
    np.asarray(pcd_grid.colors)[idxu[1:], :] = color
    
    
    old_set = set()
    
    new_set = set(idxu[1:])
    different_elements = new_set-old_set-initial_set
    old_set = set(new_set) 
    
    while(different_elements):
    
                     
        
        
        for index in different_elements:
            new_center = pcd_grid.points[index]
            k_grid, k_target, idxu, idxf = knn_search_pointclouds(tree_grid, tree_target, new_center, search_radius)
            
            coords_grid = np.asarray(pcd_grid.points)[idxu]
            coords_target = np.asarray(pcd_target.points)[idxf]

            

            idx_up_grid, idx_up_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "up")
            idx_down_grid, idx_down_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "down")
            idx_left_grid, idx_left_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "left")
            idx_right_grid, idx_right_target = get_ids_in_direction(idxu, idxf, coords_grid, coords_target, new_center, "right")
            
            if (len(idx_up_target) < stop_threshold):
                new_set.update(idx_up_grid[1:])
                np.asarray(pcd_grid.colors)[idx_up_grid[1:], :] = color
            
            if (len(idx_down_target) < stop_threshold):
                new_set.update(idx_down_grid[1:])
                np.asarray(pcd_grid.colors)[idx_down_grid[1:], :] = color
            
            if (len(idx_left_target) < stop_threshold):
                new_set.update(idx_left_grid[1:])
                np.asarray(pcd_grid.colors)[idx_left_grid[1:], :] = color
            
            if (len(idx_right_target) < stop_threshold):
                new_set.update(idx_right_grid[1:])
                np.asarray(pcd_grid.colors)[idx_right_grid[1:], :] = color
        
        different_elements = new_set-old_set-initial_set
        old_set = set(new_set) 
        
    new_set = new_set.union(initial_set)
    
    return pcd_grid, new_set
        



In [299]:
def extract_void_area(pcd_grid, pcd_target, midpoints, known_points = set(),  search_radius=1, color=[0,0,1]):
    
    for points in midpoints:
        color = [0,0,1]
        void_area, known_points  = grow_void(pcd_grid, pcd_target, points, known_points, search_radius, color=color)
        o3d.visualization.draw_geometries([void_area, pcd_flat]+marker_meshes)
    return void_area, known_points

void_area, known_points = extract_void_area(uniform_pc, pcd_flat, midpoints)
o3d.visualization.draw_geometries([void_area, pcd_flat]+marker_meshes)

In [300]:
valid_area = uniform_pc.select_by_index(list(known_points))
valid_area_voxel = o3d.geometry.VoxelGrid.create_from_point_cloud(valid_area, 0.2)
o3d.visualization.draw_geometries([valid_area_voxel, pcd_flat])